In [91]:
# Imports
import numpy as np
import pandas as pd
import sympy as sp
X = [5,4,9,12,2,7.5,6.3,2.5,8.9,4.6]
Y = [0.2,1,2,1.5,0.3,1,0.8,0.6,2.3,1.2]
class Regression:
    class SimpleRegression:
        def __init__(self,X,Y):
            self.X = X
            self.Y = Y
            X = np.array(self.X)
            Y = np.array(self.Y)

            mean_X = np.mean(X)
            mean_Y = np.mean(Y)
            x = X - mean_X
            y = Y - mean_Y
            xy = x * y
            x_pow_2 = x**2

            # Estimators
            beta_2_hat = np.sum(xy)/np.sum(x_pow_2)
            self.beta2 = beta_2_hat
            beta_1_hat = mean_Y - mean_X * beta_2_hat
            self.beta1 = beta_1_hat
            
            # y_hat
            Y_hat = beta_2_hat * X + beta_1_hat
            x_symbol = sp.Symbol("x")
            self.line = beta_1_hat + beta_2_hat * x_symbol
            mean_Y_hat = np.mean(Y_hat)

            # Errors
            u_hat =  Y - Y_hat

            # Sigma

            sigma_pow_2 = np.sum(u_hat**2)/(len(x)-2)
            simga = np.sqrt(sigma_pow_2)

            # Variance
            var_beta_2_hat = sigma_pow_2 / np.sum(x_pow_2)

            # Standard deviation
            std_beta_2_hat = np.sqrt(var_beta_2_hat)

            # R2
            y_hat = Y_hat - np.mean(Y_hat)
            y_pow_2 = y**2
            
            y_hat_pow_2 = y_hat ** 2
            u_hat_pow_2 = u_hat**2

            RSS = np.sum(u_hat_pow_2)
            TSS = np.sum(y_pow_2)
            ESS = np.sum(y_hat_pow_2)

            R2 = ESS/TSS
            self.rSquared = R2
            self.n = np.size(self.X)
            # Variance of the line
            n = len(x)
            variance = (1/(n*(n-2)))*(n*np.sum(y**2)-np.sum(y)**2-((n*np.sum(X*y)-np.sum(X)*np.sum(y))**2/(n*np.sum(X**2)-np.sum(X)**2)))

        def show_line(self):
            display(self.line)

    class MultipleRegression:

        def __init__(self, X, y):
            self.X = list(X)
            self.k = len(self.X)
            self.n = len(self.X[0])
            self.y = np.array(y).reshape(-1,1)
            self.prepare()
            self.regress()
        
        def prepare(self):
            for index, x in enumerate(self.X):
                self.X[index] = np.array(x).reshape(-1,1)
            self.X_matrices = np.ones(self.X[0].shape[0]).reshape(-1,1)

            for x in self.X:
                self.X_matrices = np.concatenate((self.X_matrices, x), axis=1)
        
        def regress(self):
            self.beta = np.linalg.inv(self.X_matrices.T.dot(self.X_matrices)).dot(self.X_matrices.T).dot(self.y)
            
        def get_beta(self):
            return self.beta
        
        def get_hat_matrix(self):
            if not hasattr(self, "hat_matrix"):
                self.hat_matrix = self.X_matrices.dot(np.linalg.inv(self.X_matrices.T.dot(self.X_matrices))).dot(self.X_matrices.T)
            return self.hat_matrix

        def get_predicted(self):
            return self.get_hat_matrix().dot(self.y)
        
        def get_hat_bar_matrix(self):
            if not hasattr(self, "hat_bar_matrix"):
               self.hat_bar_matrix = np.identity(self.get_hat_matrix().shape[0]) - self.get_hat_matrix()
            return self.hat_matrix

        def get_SSR(self):
            if not hasattr(self, "SSR"):
                self.SSR = self.y.T.dot(self.get_hat_bar_matrix()).dot(self.y)
            return self.SSR
        
        def get_sigma_estimator(self):
            if not hasattr(self, "simga_estimator"):
                self.simga_estimator = self.get_SSR()/(self.n-self.k-1)
            return self.simga_estimator

        def get_MSE(self):
            if not hasattr(self, "mse"):
                for i in range(self.X_matrices.shape[1]):
                    self.mse = (self.y - self.get_predicted())**2/self.X_matrices.shape[1]
            return self.mse

        def get_se_parametres(self):
            print(self.get_sigma_estimator())
            if not hasattr(self, "se_paramters"):
                self.se_paramters = self.get_sigma_estimator() *  np.linalg.inv(self.X_matrices.T.dot(self.X_matrices))
            return self.se_paramters
            
# Multiple Example         
a = np.array([1,23,45,2,32,5]).reshape(-1,1)
b = np.array([1,23,2,52,52,5]).reshape(-1,1)
c = np.array([1,3,24,5,25,62]).reshape(-1,1)
from sklearn.linear_model import LinearRegression
lm = LinearRegression()
lm.fit(np.concatenate((a,b), axis=1),c)
print("Coefs in sklear model: " + str(lm.coef_))
print("Intercept in sklear model: " + str(lm.intercept_))
print("Predicteds in sklear model: " + str(lm.predict(np.concatenate((a,b), axis=1))))

model = Regression.MultipleRegression([a,b],c)
model.get_se_parametres()

Coefs in sklear model: [[ 0.06362651 -0.24267566]]
Intercept in sklear model: [24.31492518]
Predicteds in sklear model: [[24.13587603]
 [20.19679472]
 [26.69276684]
 [11.82304383]
 [13.73183915]
 [23.41967943]]
[[859.92799564]]


array([[ 4.61812307e+02, -9.37013683e+00, -6.65904497e+00],
       [-9.37013683e+00,  5.16814291e-01,  2.99909300e-03],
       [-6.65904497e+00,  2.99909300e-03,  2.93558280e-01]])